# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
     ---------------------------------------- 0.0/51.6 kB ? eta -:--:--
     ----------------------- ---------------- 30.7/51.6 kB ? eta -:--:--
     ---------------------------------------- 51.6/51.6 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/990.0 kB ? eta -:--:--
   - ------------------------------------- 41.0/990.0 kB 991.0 kB/s eta 0:00:01
   --- ------------------------------------ 92.2/990.0 kB 1.1 MB/s eta 0:00:01
   --- ------------------------------------ 92.2/990.0 kB 1.1 MB/s eta 0:00:01
   ------ ------------------------------- 174.1/990.0 kB 952.6 kB/s eta 0:00:01
   ------- ------------------------------ 204.8/990.0 kB 958.4 kB/s eta 0:00:01
   --------- ------------------------------ 245.8/990.0 kB 1.0 MB/s eta 0:00:01
   ------------- -------------------------- 337.9/990.0 kB 1.0 MB/s eta 0:00:01
   -------------- ----------------------- 368.6/990.0 kB 995.6 kB/s eta 0:0

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [3]:
pip install langchain_openai

     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     -------------------------------------- 42.0/42.0 kB 675.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/46.7 kB ? eta -:--:--
   -------------------------- ------------- 30.7/46.7 kB ? eta -:--:--
   ---------------------------------------- 46.7/46.7 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/799.3 kB ? eta -:--:--
   -- ------------------------------------- 41.0/799.3 kB ? eta -:--:--
   --- ------------------------------------ 61.4/799.3 kB 3.2 MB/s eta 0:00:01
   --- ----------------------------------- 71.7/799.3 kB 787.7 kB/s eta 0:00:01
   ------- -------------------------------- 153.6/799.3 kB 1.0 MB/s eta 0:00:01
   ---------- ----------------------------- 204.8/799.3 kB 1.2 MB/s eta 0:00:01
   --------------- ------------------------ 307.2/799.3 kB 1.3 MB/s eta 0:00:01
   -----------

In [4]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [7]:
from dotenv import load_dotenv
import os 

load_dotenv()
base_url = os.getenv('BASE_URL')
api_key = os.getenv('API_KEY')

In [24]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


biology_template = """你是一位很棒的生物学老师。你擅长回答生物学问题。
之所以如此出色，是因为你能够将复杂的概念分解成各个组成部分，
先解释这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


computer_template = """你是一位很棒的计算机老师。你擅长回答计算机相关的问题。
之所以如此出色，是因为你能够将复杂的问题分解成各个组成部分，
先解释这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


chinese_template = """你是一位很棒的语文老师。你擅长回答语文相关的问题。
之所以如此出色，是因为你能够将复杂的问题分解成各个组成部分，
先解释这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

In [25]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言",
        "description": "适用于回答汉语言问题",
        "prompt_template": chinese_template,
    },
]

In [26]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", base_url=base_url, api_key=api_key)

In [27]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [28]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [29]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [30]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [31]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物学问题', '计算机: 适用于回答计算机问题', '汉语言: 适用于回答汉语言问题']


In [32]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题


In [33]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [34]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题

<< INPUT >>
{input}

<< O

In [35]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [36]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？'}
> Finished chain.
{'input': '黑体辐射是什么？', 'text': '\n\n黑体辐射是指一个完美吸收所有光波的物体，在一定温度下会发出特定的光谱分布的电磁辐射。它是理想化的概念，用来研究热辐射的特性。'}


In [37]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': 'What is the first prime number greater than 40 that is one more than a multiple of 3?'}
> Finished chain.
{'input': 'What is the first prime number greater than 40 that is one more than a multiple of 3?', 'text': '\n\n第一个大于40且比3的倍数多1的质数是43。'}


In [38]:
router_chain.verbose = True

In [39]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是宇宙中一种非常特殊的天体，它具有非常强大的引力，甚至连光都无法逃离它的吸引力。它的形成是由于质量非常大的恒星在死亡时发生的坍缩，形成一个非常紧密且密度极高的物体。黑洞的边界称为“事件视界”，在这个边界内的一切事物，包括光，都会被吸入黑洞。因为黑洞无法被直接观测到，所以我们只能通过观测其周围物质运动和辐射来推测其存在。黑洞是宇宙中最神秘和具有挑战性的物体之一，它们也是研究宇宙和宇宙规律的重要对象。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [40]:
print(
    chain.invoke(
        "细胞核是什么？"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '细胞核是什么？'}
> Finished chain.
{'input': '细胞核是什么？', 'text': '\n\n细胞核是细胞内的一个重要结构，它是一个膜包裹的细胞器官，包含着细胞的遗传物质DNA。细胞核的主要功能是控制细胞的生长、分裂和遗传信息的传递。它还通过调控基因表达来控制细胞的特征和功能。细胞核内还含有核糖核蛋白颗粒，参与蛋白质的合成。细胞核的结构和功能对细胞的生存和繁殖至关重要。'}


In [41]:
print(
    chain.invoke(
        "计算机组成原理学科有什么概念？"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '计算机组成原理学科有什么概念？'}
> Finished chain.
{'input': '计算机组成原理学科有什么概念？', 'text': '\n计算机组成原理是计算机科学的一门基础学科，它研究计算机系统的设计、构建和运行原理。它涉及到计算机硬件、软件以及它们之间的交互关系。主要包括以下几个概念：\n\n1. 计算机体系结构：指的是计算机硬件和软件之间的交互结构，包括处理器、存储器、输入输出设备以及系统总线等组件。\n\n2. 计算机指令集架构：指的是计算机系统的指令集合，包括指令的格式、指令的功能和计算机执行指令的方式等。\n\n3. 计算机中央处理器（CPU）：是计算机的核心部件，负责执行指令、控制计算机的操作和处理数据。\n\n4. 存储器：是计算机中'}


In [42]:
print(
    chain.invoke(
        "在古诗词中李白是谁"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言: {'input': '李白是谁'}
> Finished chain.
{'input': '李白是谁', 'text': '？\n\n解答：\n李白是中国唐代伟大的诗人，被称为“诗仙”。他的诗歌风格豪放、奔放，充满着对自然和生活的热爱。他的诗作受到广泛的欢迎和赞赏，被人们称为“李白诗风”。他的代表作品有《静夜思》、《蜀道难》等。他的诗歌成为了中国古典文学中不可或缺的一部分，对后世诗人也有深远的影响。'}
